[View in Colaboratory](https://colab.research.google.com/github/TonyIssacJames/EIP_public/blob/master/DNST_CIFAR10_AUG.ipynb)

In [1]:
# https://keras.io/
#!pip install -q keras
import keras

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 24
compression = 0.7
dropout_rate = 0.20

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
import numpy as np

x_train = np.append(x_train, x_train,axis=0)
y_train = np.append(y_train, y_train,axis=0)
print(x_train.shape)
#print(np.append(x_train, x_train,axis=0).shape)
#print(np.append(y_train, y_train,axis=0).shape)
print(y_train.shape)

(100000, 32, 32, 3)
(100000, 10)


In [7]:
from matplotlib import pyplot
from scipy.misc import toimage
def show_imgs(X):
    pyplot.figure(1)
    k = 0
    for i in range(0,4):
        for j in range(0,4):
            pyplot.subplot2grid((4,4),(i,j))
            pyplot.imshow(toimage(X[k]))
            k = k+1
    # show the plot
    pyplot.show()

show_imgs(x_test[:16])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  if __name__ == '__main__':


<Figure size 640x480 with 16 Axes>

In [8]:
print(y_test[:16])

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [9]:
#Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,    # randomly flip images
        fill_mode='nearest')

In [10]:
#Callback for saving the best model
from keras.callbacks import ModelCheckpoint
filepath= "weights_tr_05.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [11]:
# Dense Block
def add_denseblock(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [12]:
def add_transition(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [13]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [14]:
#num_filter = 16
#Sdropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

#Fifth_Block = add_denseblock(Fourth_Transition, num_filter, dropout_rate)
#Fifth_Transition = add_transition(Fifth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Fourth_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [15]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [17]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [18]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Epoch 1/50
782/782 [==============================] - 543s 695ms/step - loss: 1.6341 - acc: 0.3966 - val_loss: 3.2058 - val_acc: 0.3119

Epoch 00001: val_acc improved from -inf to 0.31190, saving model to weights_tr_05.best.hdf5
Epoch 2/50
782/782 [==============================] - 459s 587ms/step - loss: 1.2740 - acc: 0.5411 - val_loss: 1.3895 - val_acc: 0.5213

Epoch 00002: val_acc improved from 0.31190 to 0.52130, saving model to weights_tr_05.best.hdf5
Epoch 3/50
782/782 [==============================] - 459s 587ms/step - loss: 1.1193 - acc: 0.6007 - val_loss: 1.5700 - val_acc: 0.5509

Epoch 00003: val_acc improved from 0.52130 to 0.55090, saving model to weights_tr_05.best.hdf5
Epoch 4/50
782/782 [==============================] - 459s 588ms/step - loss: 1.0186 - acc: 0.6399 - val_loss: 1.3176 - val_acc: 0.5970

Epoch 00004: val_acc improved from 0.55090 to 0.59700, saving model to weights_tr_05.best.hdf5
Epoch 5/50
782/782 [==============================] - 459s 587ms/step - los

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



782/782 [==============================] - 459s 587ms/step - loss: 0.4240 - acc: 0.8528 - val_loss: 0.4914 - val_acc: 0.8526

Epoch 00037: val_acc improved from 0.83990 to 0.85260, saving model to weights_tr_05.best.hdf5
Epoch 38/50
782/782 [==============================] - 459s 587ms/step - loss: 0.4159 - acc: 0.8560 - val_loss: 0.5344 - val_acc: 0.8504

Epoch 00038: val_acc did not improve from 0.85260
Epoch 39/50
782/782 [==============================] - 459s 587ms/step - loss: 0.4139 - acc: 0.8582 - val_loss: 0.6556 - val_acc: 0.8211

Epoch 00039: val_acc did not improve from 0.85260
Epoch 40/50
782/782 [==============================] - 459s 587ms/step - loss: 0.4062 - acc: 0.8606 - val_loss: 0.5469 - val_acc: 0.8411

Epoch 00040: val_acc did not improve from 0.85260
Epoch 41/50
627/782 [=======================>......] - ETA: 1:28 - loss: 0.4042 - acc: 0.8616

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



782/782 [==============================] - 459s 587ms/step - loss: 0.3971 - acc: 0.8628 - val_loss: 0.5692 - val_acc: 0.8390

Epoch 00043: val_acc did not improve from 0.87200
Epoch 44/50
782/782 [==============================] - 459s 587ms/step - loss: 0.3908 - acc: 0.8646 - val_loss: 0.4330 - val_acc: 0.8681

Epoch 00044: val_acc did not improve from 0.87200
Epoch 45/50
782/782 [==============================] - 459s 587ms/step - loss: 0.3889 - acc: 0.8658 - val_loss: 0.4572 - val_acc: 0.8669

Epoch 00045: val_acc did not improve from 0.87200
Epoch 46/50
782/782 [==============================] - 459s 587ms/step - loss: 0.3726 - acc: 0.8728 - val_loss: 0.4904 - val_acc: 0.8545

Epoch 00049: val_acc did not improve from 0.87200
Epoch 50/50
782/782 [==============================] - 459s 587ms/step - loss: 0.3711 - acc: 0.8720 - val_loss: 0.5077 - val_acc: 0.8528

Epoch 00050: val_acc did not improve from 0.87200


In [19]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 18s 2ms/step
Test loss: 0.5077479893147946
Test accuracy: 0.8528


In [20]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_tr_05.h5")
print("Saved model to disk")

Saved model to disk


In [22]:
from keras.models import load_model

new_model = load_model("weights_tr_05.best.hdf5")
score = new_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 22s 2ms/step
Test loss: 0.41088400869369507
Test accuracy: 0.872


In [0]:
#from google.colab import files

#files.download('DNST_model.h5')